In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Info: Precompiling ProfileView [c46f51b8-102a-5cf2-8d2c-8597cb0e0da7]
└ @ Base loading.jl:1278
Gtk-Message: 22:49:56.647: Failed to load module "canberra-gtk-module"
Gtk-Message: 22:49:56.647: Failed to load module "canberra-gtk-module"
Gtk-Message: 22:50:00.921: Failed to load module "canberra-gtk-module"
Gtk-Message: 22:50:00.922: Failed to load module "canberra-gtk-module"
Gtk-Message: 22:50:13.654: Failed to load module "canberra-gtk-module"
Gtk-Message: 22:50:13.655: Failed to load module "canberra-gtk-module"


In [3]:
Threads.nthreads()

1

### Integration

In [15]:
Nx = 20
Ny = 1
m = 20
Ne = 300
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [16]:
M = HermiteMap(m, X);

In [18]:
@time optimize(M, X, 10; P = serial);

  1.179350 seconds (968.46 k allocations: 1.278 GiB, 5.46% gc time)


In [21]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = serial)
    end
end

timing (generic function with 1 method)

In [22]:
timing();

  865.551 ms (968774 allocations: 1.28 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)
